In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Declaring Continuous Variables

## Calling add_input and add_output
Every component in an OpenMDAO model is an instance of `ExplicitComponent`, `ImplicitComponent`, or a subclass of one of these classes. Regardless of the type, each component has input variables and output variables that it must declare.

In explicit and implicit components, the user must call `add_input` and `add_output` to declare variables in the `setup` method. An example is given below.

In [ ]:
import openmdao.api as om


class TestExplCompSimple(om.ExplicitComponent):

    def setup(self):
        self.add_input('length', val=1., desc='length of rectangle')
        self.add_input('width', val=1., desc='width of rectangle')
        self.add_output('area', val=1., desc='area of rectangle')

    def setup_partials(self):
        self.declare_partials('*', '*')

    def compute(self, inputs, outputs):
        outputs['area'] = inputs['length'] * inputs['width']


In [ ]:
"""Just to run the code for TestExplCompSimple."""
from openmdao.utils.assert_utils import assert_near_equal

p = om.Problem(model=TestExplCompSimple())
p.setup()

assert_near_equal(p.get_val('length'), 1.)

```{note}
Variable names have few restrictions, but the following characters are not allowed in a variable name: ‘.’, ‘*’, ‘?’, ‘!’, ‘[‘, ‘]’.
```


## Method Signatures

```{eval-rst}
    .. automethod:: openmdao.core.component.Component.add_input
        :noindex:
```
```{eval-rst}
    .. automethod:: openmdao.core.component.Component.add_output
        :noindex:
```

## Usage

1\. Declaring with only the default value. 


In [ ]:
import numpy as np

class CompAddWithDefault(om.ExplicitComponent):
    """Component for tests for declaring only default value."""

    def setup(self):
        self.add_input('x_a')
        self.add_input('x_b', val=3.)
        self.add_input('x_c', val=(3., 3.))
        self.add_input('x_d', val=[3., 3.])
        self.add_input('x_e', val=3. * np.ones((2, 2)))

        self.add_output('y_a')
        self.add_output('y_b', val=6.)
        self.add_output('y_c', val=(6., 6., 6.))
        self.add_output('y_d', val=[6., 6., 6.])
        self.add_output('y_e', val=6. * np.ones((3, 2)))

In [ ]:
"""Test declaring only default value."""
p = om.Problem(model=CompAddWithDefault())
p.setup()

print(p.get_val('x_a'))

In [ ]:
print(p.get_val('x_b'))

In [ ]:
print(p.get_val('x_c'))

In [ ]:
print(p.get_val('x_d'))

In [ ]:
print(p.get_val('x_e'))

In [ ]:
print(p.get_val('y_a'))

In [ ]:
print(p.get_val('y_b'))

In [ ]:
print(p.get_val('y_c'))

In [ ]:
print(p.get_val('y_d'))

In [ ]:
print(p.get_val('y_e'))

In [ ]:
assert_near_equal(p.get_val('x_a'), 1.)
assert_near_equal(p.get_val('x_b'), 3.)
assert_near_equal(p.get_val('x_c'), 3. * np.ones(2))
assert_near_equal(p.get_val('x_d'), 3. * np.ones(2))
assert_near_equal(p.get_val('x_e'), 3. * np.ones((2, 2)))
assert_near_equal(p.get_val('y_a'), 1.)
assert_near_equal(p.get_val('y_b'), 6.)
assert_near_equal(p.get_val('y_c'), 6. * np.ones(3))
assert_near_equal(p.get_val('y_d'), 6. * np.ones(3))
assert_near_equal(p.get_val('y_e'), 6. * np.ones((3, 2)))

2\. Declaring with only the *shape* argument.


In [ ]:
class CompAddWithShape(om.ExplicitComponent):
    """Component for tests for declaring only shape."""

    def setup(self):
        self.add_input('x_a', shape=2)
        self.add_input('x_b', shape=(2, 2))
        self.add_input('x_c', shape=[2, 2])

        self.add_output('y_a', shape=3)
        self.add_output('y_b', shape=(3, 3))
        self.add_output('y_c', shape=[3, 3])

In [ ]:
"""Test declaring only shape."""
p = om.Problem(model=CompAddWithShape())
p.setup()

print(p.get_val('x_a'))

In [ ]:
print(p.get_val('x_b'))

In [ ]:
print(p.get_val('x_c'))

In [ ]:
print(p.get_val('y_a'))

In [ ]:
print(p.get_val('y_b'))

In [ ]:
print(p.get_val('y_c'))

In [ ]:
assert_near_equal(p.get_val('x_a'), np.ones(2))
assert_near_equal(p.get_val('x_b'), np.ones((2, 2)))
assert_near_equal(p.get_val('x_c'), np.ones((2, 2)))
assert_near_equal(p.get_val('y_a'), np.ones(3))
assert_near_equal(p.get_val('y_b'), np.ones((3, 3)))
assert_near_equal(p.get_val('y_c'), np.ones((3, 3)))


4\. Declaring an array variable with a scalar default value.


In [ ]:
class CompAddArrayWithScalar(om.ExplicitComponent):
    """Component for tests for declaring a scalar val with an array variable."""

    def setup(self):
        self.add_input('x_a', val=2.0, shape=(6))
        self.add_input('x_b', val=2.0, shape=(3, 2))
        self.add_output('y_a', val=3.0, shape=(6))
        self.add_output('y_b', val=3.0, shape=(3, 2))

In [ ]:
"""Test declaring a scalar val with an array variable."""
p = om.Problem(model=CompAddArrayWithScalar())
p.setup()

In [ ]:
print(p.get_val('x_a'))

In [ ]:
print(p.get_val('x_b'))

In [ ]:
print(p.get_val('y_a'))

In [ ]:
print(p.get_val('y_b'))

In [ ]:
assert_near_equal(p.get_val('x_a'), 2. * np.ones(6))
assert_near_equal(p.get_val('x_b'), 2. * np.ones((3, 2)))
assert_near_equal(p.get_val('y_a'), 3. * np.ones(6))
assert_near_equal(p.get_val('y_b'), 3. * np.ones((3, 2)))

6\. Declaring an output with bounds, using *upper* and/or *lower* arguments.


In [ ]:
class CompAddWithBounds(om.ExplicitComponent):
    """Component for tests for declaring bounds."""

    def setup(self):
        self.add_input('x')

        self.add_output('y_a', val=2.0, lower=0.)
        self.add_output('y_b', val=2.0, lower=0., upper=10.)
        self.add_output('y_c', val=2.0 * np.ones(6),  lower=np.zeros(6), upper=10.)
        self.add_output('y_d', val=2.0 * np.ones(6), lower=0., upper=[12, 10, 10, 10, 10, 12])
        self.add_output('y_e', val=2.0 * np.ones((3, 2)), lower=np.zeros((3, 2)))

In [ ]:
"""Test declaring bounds."""
p = om.Problem(model=CompAddWithBounds())
p.setup()

print(p.get_val('y_a'))

In [ ]:
print(p.get_val('y_b'))

In [ ]:
print(p.get_val('y_c'))

In [ ]:
print(p.get_val('y_d'))

In [ ]:
print(p.get_val('y_e'))

In [ ]:
assert_near_equal(p.get_val('y_a'), 2.)
assert_near_equal(p.get_val('y_b'), 2.)
assert_near_equal(p.get_val('y_c'), 2. * np.ones(6))
assert_near_equal(p.get_val('y_d'), 2. * np.ones(6))
assert_near_equal(p.get_val('y_e'), 2. * np.ones((3, 2)))

7\. Adding tags to input and output variables. These tags can then be used to filter what gets returned from the `System.get_io_metadata` method and displayed/returned from the `System.list_inputs` and `System.list_outputs` methods.



In [ ]:
class RectangleCompWithTags(om.ExplicitComponent):
    """
    A simple Explicit Component that also has input and output with tags.
    """

    def setup(self):
        self.add_input('length', val=1., tags=["tag1", "tag2"])
        self.add_input('width', val=1., tags=["tag2"])
        self.add_output('area', val=1., tags="tag1")

    def setup_partials(self):
        self.declare_partials('*', '*')

    def compute(self, inputs, outputs):
        outputs['area'] = inputs['length'] * inputs['width']

prob = om.Problem(RectangleCompWithTags())
prob.setup(check=False)
prob.run_model()

# Inputs no tags
inputs = prob.model.list_inputs(val=False, out_stream=None)
print(sorted(inputs))

In [ ]:
# Inputs with tags
inputs = prob.model.list_inputs(val=False, out_stream=None, tags="tag1")
print(sorted(inputs))

In [ ]:
# Inputs with multiple tags
inputs = prob.model.list_inputs(val=False, out_stream=None, tags=["tag1", "tag2"])
print(sorted(inputs))

In [ ]:
# Inputs with tag that does not match
inputs = prob.model.list_inputs(val=False, out_stream=None, tags="tag3")
print(sorted(inputs))

In [ ]:
# Outputs no tags
outputs = prob.model.list_outputs(val=False, out_stream=None)
print(sorted(outputs))

In [ ]:
# Outputs with tags
outputs = prob.model.list_outputs(val=False, out_stream=None, tags="tag1")
print(sorted(outputs))

In [ ]:
# Outputs with multiple tags
outputs = prob.model.list_outputs(val=False, out_stream=None, tags=["tag1", "tag3"])
print(sorted(outputs))

In [ ]:
# Outputs with tag that does not match
outputs = prob.model.list_outputs(val=False, out_stream=None, tags="tag3")
print(sorted(outputs))

In [ ]:
# Inputs no tags
inputs = prob.model.list_inputs(val=False, out_stream=None)
assert(sorted(inputs) == [
    ('length', {}),
    ('width', {}),
])

# Inputs with tags
inputs = prob.model.list_inputs(val=False, out_stream=None, tags="tag1")
assert(sorted(inputs) == [
    ('length', {}),
])

# Inputs with multiple tags
inputs = prob.model.list_inputs(val=False, out_stream=None, tags=["tag1", "tag2"])
assert(sorted(inputs) == [
    ('length', {}),
    ('width', {}),
])

# Inputs with tag that does not match
inputs = prob.model.list_inputs(val=False, out_stream=None, tags="tag3")
assert(sorted(inputs) == [])

# Outputs no tags
outputs = prob.model.list_outputs(val=False, out_stream=None)
assert(sorted(outputs) == [
    ('area', {}),
])

# Outputs with tags
outputs = prob.model.list_outputs(val=False, out_stream=None, tags="tag1")
assert(sorted(outputs) == [
    ('area', {}),
])

# Outputs with multiple tags
outputs = prob.model.list_outputs(val=False, out_stream=None, tags=["tag1", "tag3"])
assert(sorted(outputs) == [
    ('area', {}),
])

# Outputs with tag that does not match
outputs = prob.model.list_outputs(val=False, out_stream=None, tags="tag3")
assert(sorted(outputs) == [])
